![header](https://d25m59h0ya0u4t.cloudfront.net/pub/uploadedImages/3179520201006_ML_JuliaContentTT%20with%20language-100.jpg)

During this session you'll get to know Julia's powerful core design, the Multiple Dispatch and learn more about Flux.jl in order to do state-of-the-art machine learning with just a few lines of code. 

What you will learn
* How to use Julia and **Multiple Dispatch** as well as **Flux.jl**
* learn more on **scientific machine learning**

-------

# PART I - MULTIPLE DISPATCH

This section was adapted from a famous [julia talk about reusability](https://www.youtube.com/watch?reload=9&reload=9&v=kc9HwsxE1OY)

## Custom Types

In julia you can easily define your custom type hierarchy.

A notable difference to other type systems is that abstract types in julia do not have any attributes, but only serve to define the hierarchy.

In [ ]:
abstract type Pet end
struct Dog <: Pet
    name::String
end
# TODO define Cat

We can instantiate the struct types by simple call syntax. They come with a default constructor.

In [ ]:
fido     = Dog("Fido")
rex      = Dog("Rex")
# TODO create two cats
whiskers = ...
spots    = ...

## Defining Generic Functions

In julia you can define functions assuming whatever other functions you want. All contracts/interfaces are guaranteed by convention/documentation.

For instance the following `encounter` assumes that `meets` and `names` are defined for the arguments.

In [ ]:
function encounter(a::Pet, b::Pet)
    verb = meets(a, b)
    println("$(name(a)) meets $(name(b)) and $verb")
end

You don't have to specify types. If so, the type defaults to `Any`, which includes everything.

In [ ]:
name(a) = a.name  # short one-line syntax for defining functions 

## Multiple Dispatch

In Julia you can specialize your functions for any combination of types. Julia will always choose the most specific implementation available. This is called **Multiple Dispatch**.

In [ ]:
meets(a::Pet, b::Pet) = "does nothing"

encounter(fido, rex)

In [ ]:
# TODO
# define your own implementations of meets between Cats, Dogs and Pets

meets(a::Pet, b::Pet) = ...
...

In [ ]:
encounter(fido, rex)
encounter(fido, whiskers)
encounter(whiskers, rex)
encounter(whiskers, spots)

You can always add new types and extend functions. This even holds true for functions defined in other packages.

In [ ]:
struct TheDuck <: Pet end
name(::TheDuck) = "The duck"

theduck = TheDuck()

In [ ]:
encounter(theduck, rex)

## Fully Flexible Arrays

In [ ]:
pets = [fido, rex, whiskers, spots, theduck]

In [ ]:
name.(pets)  # use dot . to apply an arbitrary function elementwise

In [ ]:
[meets(a, b) for a in pets, b in pets]  # multi dimensional for-comprehension

-------

# PART II - SCIENTIFIC MACHINE LEARNING with Flux.jl

In [ ]:
using Flux  # takes about a minute when run the first time

Let me cite Flux.ml:

"""

**Flux: The Julia Machine Learning Library**

Flux is a library for machine learning. It comes "batteries-included" with many useful tools built in, but also lets you use the full power of the Julia language where you need it. We follow a few key principles:

* **Doing the obvious thing.** Flux has relatively few explicit APIs for features like regularisation or embeddings. Instead, writing down the mathematical form will work – and be fast.
* **You could have written Flux.** All of it, from LSTMs to GPU kernels, is straightforward Julia code. When in doubt, it’s well worth looking at the source. If you need something different, you can easily roll your own.
* **Play nicely with others.** Flux works well with Julia libraries from data frames and images to differential equation solvers, so you can easily build complex data processing pipelines that integrate Flux models.

"""

## Basics - Taking Gradients

In [ ]:
f(x) = 3x^2 + 2x + 1

In [ ]:
# df/dx = 6x + 2
f'(2)  # after `using Flux` you have access to automatic differentiation of arbitrary functions (actually this is given by Zygote.jl which is a subpackage of the Flux eco system)

In [ ]:
df(x) = gradient(f, x)[1]
df(2)

In [ ]:
# TODO compute second derivative

You may ask how far does this go? Can everything be autodifferentiated? Actually almost everything, including arbitrary controlflows, recursions, loops, and even mutable datastructures. See https://fluxml.ai/Zygote.jl/latest/#Taking-Gradients-1 for details.

## Controlling a Trebuchet

![trebuchet](https://fluxml.ai/assets/2019-03-05-dp-vs-rl/trebuchet-basic.gif)

There is Trebuchet, which throws a mass to a target. The mass is to be
released at an angle, and at certain velocity so that it lands on the target.
The velocity of release is determined by the counterweight of the Trebuchet.
Given conditions of environment we are required to predict the angle of
release and counterweight.

* **Input:**  Wind speed,   Target distance
* **Output:** ReleaseAngle, Weight

![overview](https://fluxml.ai/assets/2019-03-05-dp-vs-rl/trebuchet-flow.png)

In [ ]:
using Flux
import Zygote
using Random
import Trebuchet
using Plots
plotlyjs()

In [ ]:
function visualize_trebuchet(;target=100, wind_speed=1.0, release_angle=45, weight=98.09)  # default values from TrebuchetState
    # state is going to be mutated by simulate, hence we capsulate it into our own method
    release_angle = Trebuchet.deg2rad(release_angle)
    state = Trebuchet.TrebuchetState(wind_speed=wind_speed, release_angle=release_angle, weight=weight)
    Trebuchet.simulate(state)  # should be named `simulate!(t)`
    Trebuchet.visualise(state, target)
end 

function shoot_trebuchet(;wind_speed=1.0, release_angle=45, weight=98.09)
    release_angle = Trebuchet.deg2rad(release_angle)
    state = Trebuchet.TrebuchetState(;wind_speed=wind_speed, release_angle=release_angle, weight=weight)
    weight > 0 || return 0.0
    Trebuchet.simulate(state)
    Trebuchet.endDist(state)
end

In [ ]:
visualize_trebuchet(target=50)

In [ ]:
shoot_trebuchet()

## Create a Model

In [ ]:
Random.seed!(0)
model = Chain(Dense(2, 16, σ),
              Dense(16, 64, σ),
              Dense(64, 16, σ),
              Dense(16, 2))
θ = params(model)

In [ ]:
function aim(wind, target)
  angle, weight = model([wind, target])
  angle = σ(angle)*90
  weight = weight + 200
  (release_angle=angle, weight=weight)
end

In [ ]:
function visualize_model(;wind_speed=1.0, target=100)
    release_angle, weight = aim(wind_speed, target)
    visualize_trebuchet(target=target, wind_speed=wind_speed, release_angle=release_angle, weight=weight)
end

function shoot_model(;wind_speed=1.0, target=100)
    release_angle, weight = aim(wind_speed, target)
    # shoot_trebuchet uses array mutation internally, which is not yet supported by Zygote ReverseDiff
    # however forwarddiff works with everything, including array mutation and try/catch,
    # hence we mark this respectively
    Zygote.forwarddiff([wind_speed, release_angle, weight]) do (wind_speed, release_angle, weight)
        shoot_trebuchet(wind_speed=wind_speed, release_angle=release_angle, weight=weight)
    end
end

In [ ]:
# TODO execute visualize_model

In [ ]:
# TODO execute shoot_model

## Do Syntax - a tiny excurse to special Julia syntax

If we have a function which takes another function as the first argument, for instance ...

In [ ]:
apply_function(f, args...) = f(args...)

Then we can just pass a function, for instance an adhoc anonymous function ...

In [ ]:
apply_function((a, b) -> a + b, 1, 2)

or we can use the equivalent `do` syntax

In [ ]:
apply_function(1, 2) do a, b
    a + b
end

## Train

Finally we want to train our model to become better at shooting the trebuchet.

In [ ]:
target_min, target_max = 20, 100	# Maximum target distance
wind_speed_mean = 5 # Maximum wind speed

# linear interpolation helper
lerp(x, lo, hi) = x*(hi-lo)+lo

random_target() = (
    wind_speed = randn() * wind_speed_mean,
    target = lerp(rand(), target_min, target_max)
)

In [ ]:
losses = Float64[]
iterations = Int[]
i = 0

In [ ]:
optimizer = ADAM()
try
    while true
        i += 1
        wind_speed, target = random_target()
        ∇θ = gradient(θ) do
            hit = shoot_model(wind_speed=wind_speed, target=target)
            loss = (hit - target)^2
            Zygote.ignore() do
                if i % 100 == 0
                    push!(losses, loss)
                    push!(iterations, i)
                    plot(iterations, losses, show = :inline, yscale = :log10,
                        label = "square-loss", xlabel = "#iteration", ylabel="loss (log10 scale)")
                end
            end
            loss
        end
        Flux.update!(optimizer, θ, ∇θ)
    end
    
catch e
    if e isa InterruptException
        visualize_model(;random_target()...)
    end
end 

----------

# Thank you for participating

In case of any questions feel free to reach me at s.sahm@reply.de

If you are curious for more or want to do a Julia project, just tell me. I am always glad about new enthusiasts.

<img src="https://julialang.org/assets/infra/logo.svg" alt="JuliaLogo" width="30%" align="right"/>

<img src="https://3gp10c1vpy442j63me73gy3s-wpengine.netdna-ssl.com/wp-content/uploads/2020/01/Machine-Learning-Reply-LOGO-150dpi-600x149.jpg" alt="MachineLearningReply" width="30%" align="right"/>